# New outbreaks in Germany?

In [ ]:
import oscovida as osc
import pandas as pd
pd.options.display.max_rows = 100
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [ ]:
x = osc.fetch_data_germany(include_last_day=True)

In [ ]:
def cumulative_cases(x):
    cases_diff = x.groupby(['Landkreis', 'Meldedatum']).agg('sum')['AnzahlFall']
    # cases_diff has a multi-index
    
    # sanity check: what is the total number of cases
    total = cases_diff.sum()
    
    # Turn into table with row index being dates, and Landkreise as Columns
    tmp1 = cases_diff.unstack().T
    
    # compute cumulative sum per Landkreis
    tmp2 = tmp1.cumsum()
    
    # replace NaNs from alignment (data not available for all dates)
    tmp3 = tmp2.fillna(method='pad')
    
    # We may have some NaNs left for the earliest dates. Replace by zeros
    tmp4 = tmp3.fillna(value=0)
    
    # Compute sum again. As we have cumulative numbers, we only take the last row:
    total2 = tmp4.iloc[-1, :].sum()

    diff = abs(total - total2)
    if diff > 0:
        print("diff=", diff)
    assert diff == 0
    
    # make index a proper datetime object
    tmp4.index = pd.to_datetime(tmp4.index)
    
    return tmp4

In [ ]:
y = cumulative_cases(x)

In [ ]:
#y

In [ ]:
y.diff().iloc[-1,:].sum()

In [ ]:
def get_7_day_averages(y):
    y = y.copy()
    total = y.iloc[-1,:].sum()
    print(total)
    diffs = y.diff()
    print(diffs.sum().sum())
    # diffs2 = diffs.rolling(7, center=True).mean()
    diffs2 = diffs
    diffs2 = diffs.rolling(7, center=True).mean()
    # This introduced new NaNs at the beginning and end. Get rid of those
    len1 = len(diffs2)
    diffs2.dropna(inplace=True)
    len2 = len(diffs2)
    assert len1 - len2 == 7
    print(diffs2.sum().sum())

    # sample every 7 days, but start from most recent data
    data = diffs2
    print(data.sum().sum())
    end_time = data.index[-1]
    data['time to end'] = end_time - data.index
    data['Meldedatum'] = data.index
    data.set_index('time to end', inplace=True)
    print(data.sum().sum())

    data2 = data.resample('1D').asfreq() # Or your function
    # data2 = data
    print(data2.sum().sum()*1)

    data2['datetime'] = end_time - data2.index
    data2.set_index('datetime', inplace=True)
    
    return data2

In [ ]:
z = get_7_day_averages(y)
z.drop(columns='Meldedatum', inplace=True)

In [ ]:
z.iloc[1,:].index

In [ ]:
df.drop('Meldedatum', inplace=True)

In [ ]:
df = pd.DataFrame()
df['last week'] = z.iloc[0, :]
# df['prev weeks'] = z.iloc[1:3].mean()
df['prev weeks'] = z.iloc[1, :]

In [ ]:
df['growth'] = df['last week'] / df['prev weeks']

In [ ]:
df

In [ ]:
df.sort_values(by='last week', ascending=False).head(40)

In [ ]:
df.loc["LK Rhein-Sieg-Kreis"]

In [ ]:
df.sort_values(by='growth', ascending=False).head(40)

In [ ]:
df2 = df.drop(df[df['last week'] < 2].index)

In [ ]:
df2.shape

In [ ]:
df.shape

In [ ]:
df2.sort_values(by='growth', ascending=False).head(40)

In [ ]:
!pwd